<a href="https://www.kaggle.com/code/shedai/llm-evaluation?scriptVersionId=256554953" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## LLM Değerlendirme Metriklerini Anlama Kılavuzu

Bu notebook, Büyük Dil Modelleri'ni (LLM) değerlendirmek için kullanılan temel metrikleri pratik olarak anlamanıza yardımcı olmak için tasarlanmıştır. Her bir metriği, ne işe yaradığını açıklayarak ve basit kod örnekleriyle nasıl çalıştığını göstererek ele alacağız.

## Adım 0: Gerekli Kütüphanelerin Kurulumu ve Yüklenmesi

Öncelikle, Hugging Face tarafından geliştirilen ve bu metrikleri hesaplamamızı çok kolaylaştıran kütüphaneleri kuralım.

In [2]:
# Gerekli kütüphaneleri sessizce kuruyoruz
!pip install evaluate transformers sentence-transformers scipy -q

import torch
import evaluate
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util

print("Kütüphaneler başarıyla yüklendi!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 22.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not c

2025-08-18 05:41:09.879272: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755495670.108815      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755495670.176463      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Kütüphaneler başarıyla yüklendi!


## Bölüm 1: Perplexity (Şaşkınlık) - Modelin Metni Ne Kadar İyi "Bildiği"

Perplexity, bir modelin bir metin dizisini ne kadar olası bulduğunun bir ölçüsüdür. Düşük perplexity, modelin metni "beklenen" ve "anlaşılır" bulduğu anlamına gelir. Yüksek perplexity ise modelin metni "şaşırtıcı" ve "anlamsız" bulduğunu gösterir. Genellikle modelin dilin genel yapısını ne kadar iyi öğrendiğini ölçmek için kullanılır.

In [3]:
# Hızlı çalışması için küçük bir model ve tokenizer'ını yüklüyoruz
model_id = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# "İyi" (gramatik olarak doğru ve mantıklı) bir cümle
iyi_cumle = "The quick brown fox jumps over the lazy dog."

# "Kötü" (anlamsız) bir cümle
kotu_cumle = "Fox brown quick lazy dog over the jumps."

# --- İyi Cümle İçin Hesaplama ---
# 1. Adım: Cümleyi modelin anlayacağı token ID'lerine çevir (Tokenization)
iyi_tokenlar = tokenizer(iyi_cumle, return_tensors="pt")
input_ids_iyi = iyi_tokenlar.input_ids

# 2. Adım: Modelden, bu token dizisi için kayıp (loss) değerini hesaplamasını istiyoruz.
# Loss, modelin bir sonraki token'ı tahmin ederken ne kadar "şaşırdığının" ortalamasıdır.
with torch.no_grad():
    outputs_iyi = model(input_ids=input_ids_iyi, labels=input_ids_iyi)
    loss_iyi = outputs_iyi.loss

# 3. Adım: Perplexity, loss değerinin üslü (exponential) halidir.
perplexity_iyi = torch.exp(loss_iyi)

print(f"İyi Cümle: '{iyi_cumle}'")
print(f"  -> Modelin Hesapladığı Ortalama Kayıp (Loss): {loss_iyi.item():.2f}")
print(f"  -> Perplexity (Şaşkınlık) Skoru (exp(loss)): {perplexity_iyi.item():.2f}")
print("-" * 30)


# --- Kötü Cümle İçin Hesaplama ---
# Aynı adımları kötü cümle için de tekrarlıyoruz.
kotu_tokenlar = tokenizer(kotu_cumle, return_tensors="pt")
input_ids_kotu = kotu_tokenlar.input_ids

with torch.no_grad():
    outputs_kotu = model(input_ids=input_ids_kotu, labels=input_ids_kotu)
    loss_kotu = outputs_kotu.loss

perplexity_kotu = torch.exp(loss_kotu)

print(f"Kötü Cümle: '{kotu_cumle}'")
print(f"  -> Modelin Hesapladığı Ortalama Kayıp (Loss): {loss_kotu.item():.2f}")
print(f"  -> Perplexity (Şaşkınlık) Skoru (exp(loss)): {perplexity_kotu.item():.2f}")

print("\nBeklendiği gibi, anlamsız cümlenin loss ve perplexity değerleri çok daha yüksek çıktı.")

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


İyi Cümle: 'The quick brown fox jumps over the lazy dog.'
  -> Modelin Hesapladığı Ortalama Kayıp (Loss): 6.35
  -> Perplexity (Şaşkınlık) Skoru (exp(loss)): 569.89
------------------------------
Kötü Cümle: 'Fox brown quick lazy dog over the jumps.'
  -> Modelin Hesapladığı Ortalama Kayıp (Loss): 8.26
  -> Perplexity (Şaşkınlık) Skoru (exp(loss)): 3867.44

Beklendiği gibi, anlamsız cümlenin loss ve perplexity değerleri çok daha yüksek çıktı.


Ara Çıktı Analizi:
Gördüğünüz gibi, model anlamsız cümleyi tahmin ederken çok daha fazla "şaşırdı" ve bu nedenle perplexity skoru daha yüksek oldu. Bu, modelin İngilizce dilinin temel yapısını anladığını gösterir.

## Bölüm 2: BLEU - Makine Çevirisi ve Metin Üretimi İçin

BLEU (Bilingual Evaluation Understudy), model tarafından üretilen metnin, insan tarafından yazılmış bir veya daha fazla referans metne ne kadar benzediğini ölçer. Kelime gruplarının (n-gram) eşleşmesine bakar. Genellikle 0 ile 1 arasında bir skor üretir (bazen 0-100). Yüksek skor, daha iyi eşleşme demektir.

In [4]:
# BLEU metriğini yüklüyoruz
bleu = evaluate.load("bleu")

# Örnek 1: İyi bir eşleşme
model_ciktisi_1 = "kedi paspasın üzerinde oturdu"
referans_1 = ["kedi paspasın üstünde oturdu"]
sonuc_1 = bleu.compute(predictions=[model_ciktisi_1], references=[referans_1])


# Örnek 2: Kötü bir eşleşme
model_ciktisi_2 = "orada bir kedi vardı"
referans_2 = ["kedi paspasın üstünde oturdu"]
sonuc_2 = bleu.compute(predictions=[model_ciktisi_2], references=[referans_2])

# Örnek 3: Anlam aynı, kelimeler farklı (BLEU'nun zayıflığı)
model_ciktisi_3 = "otomobil otoyolda hızla ilerliyordu"
referans_3 = ["araba yolda süratle gidiyordu"]
sonuc_3 = bleu.compute(predictions=[model_ciktisi_3], references=[referans_3])


print("--- Örnek 1: İyi Eşleşme ---")
print(f"Model Çıktısı: '{model_ciktisi_1}'")
print(f"Referans: '{referans_1[0]}'")
print(f"BLEU Skoru: {sonuc_1['bleu']:.4f}")
print("Ara Çıktılar (n-gram hassasiyetleri):", sonuc_1['precisions'])

print("\n--- Örnek 2: Kötü Eşleşme ---")
print(f"Model Çıktısı: '{model_ciktisi_2}'")
print(f"Referans: '{referans_2[0]}'")
print(f"BLEU Skoru: {sonuc_2['bleu']:.4f}")
print("Ara Çıktılar (n-gram hassasiyetleri):", sonuc_2['precisions'])


print("\n--- Örnek 3: Anlamsal Benzerlik Ama Düşük BLEU ---")
print(f"Model Çıktısı: '{model_ciktisi_3}'")
print(f"Referans: '{referans_3[0]}'")
print(f"BLEU Skoru: {sonuc_3['bleu']:.4f}")
print("Ara Çıktılar (n-gram hassasiyetleri):", sonuc_3['precisions'])

--- Örnek 1: İyi Eşleşme ---
Model Çıktısı: 'kedi paspasın üzerinde oturdu'
Referans: 'kedi paspasın üstünde oturdu'
BLEU Skoru: 0.0000
Ara Çıktılar (n-gram hassasiyetleri): [0.75, 0.3333333333333333, 0.0, 0.0]

--- Örnek 2: Kötü Eşleşme ---
Model Çıktısı: 'orada bir kedi vardı'
Referans: 'kedi paspasın üstünde oturdu'
BLEU Skoru: 0.0000
Ara Çıktılar (n-gram hassasiyetleri): [0.25, 0.0, 0.0, 0.0]

--- Örnek 3: Anlamsal Benzerlik Ama Düşük BLEU ---
Model Çıktısı: 'otomobil otoyolda hızla ilerliyordu'
Referans: 'araba yolda süratle gidiyordu'
BLEU Skoru: 0.0000
Ara Çıktılar (n-gram hassasiyetleri): [0.0, 0.0, 0.0, 0.0]


Ara Çıktı Analizi:
precisions listesi bize sırasıyla 1-gram, 2-gram, 3-gram ve 4-gram'ların ne kadarının eşleştiğini gösterir. İlk örnekte kelimeler çok benzediği için skor yüksek, ikincide ise düşük çıktı. Üçüncü örnekte ise anlam aynı olmasına rağmen farklı kelimeler kullanıldığı için BLEU skoru sıfır çıktı. Bu, BLEU'nun en büyük zayıflığıdır.

## Bölüm 3: ROUGE - Metin Özetleme İçin

ROUGE, özellikle metin özetleme görevlerinde kullanılır. BLEU'nun tersine, referans metindeki kelimelerin ne kadarının model çıktısında yakalandığına odaklanır (Recall odaklı). Farklı türleri vardır:

* ROUGE-1 / ROUGE-2: Tekli kelimelerin (unigram) / ikili kelimelerin (bigram) ne kadarının yakalandığı.

* ROUGE-L: En uzun ortak alt diziyi (Longest Common Subsequence) bularak yapısal benzerliği ölçer.

In [5]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8812f492fab047bf9c681a64871efb0d85b7cfa5c241b8af37a5b46f62464b37
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [6]:
# ROUGE metriğini yüklüyoruz
# Kurulum hücresine 'rouge_score' eklediğimiz için bu artık çalışacak.
rouge = evaluate.load('rouge')

model_ozeti = "Uzaylılar Dünya'ya indi ve liderler barışçıl bir görüşme yaptı."
referans_ozet = "Liderler, Dünya'ya iniş yapan uzaylılarla barışçıl bir ilk temas kurdu."

sonuclar = rouge.compute(predictions=[model_ozeti], references=[referans_ozet])

print(f"Model Özeti: '{model_ozeti}'")
print(f"Referans Özet: '{referans_ozet}'")
print("\n--- ROUGE Skorları ---")
print(f"ROUGE-1 (Tekli kelime eşleşmesi): {sonuclar['rouge1']:.4f}")
print(f"ROUGE-2 (İkili kelime eşleşmesi): {sonuclar['rouge2']:.4f}")
print(f"ROUGE-L (En uzun ortak alt dizi): {sonuclar['rougeL']:.4f}")

Model Özeti: 'Uzaylılar Dünya'ya indi ve liderler barışçıl bir görüşme yaptı.'
Referans Özet: 'Liderler, Dünya'ya iniş yapan uzaylılarla barışçıl bir ilk temas kurdu.'

--- ROUGE Skorları ---
ROUGE-1 (Tekli kelime eşleşmesi): 0.5517
ROUGE-2 (İkili kelime eşleşmesi): 0.2963
ROUGE-L (En uzun ortak alt dizi): 0.4138


Ara Çıktı Analizi:
ROUGE bize farklı seviyelerde analiz sunar. ROUGE-1 skoru "Uzaylılar", "Dünya'ya", "indi", "liderler", "barışçıl" gibi ortak kelimeler sayesinde görece yüksek çıkarken, ROUGE-2 skoru "barışçıl bir" gibi ortak ikili kelime grupları az olduğu için daha düşüktür. ROUGE-L ise cümlelerin genel akışının ne kadar benzediğini gösterir.

## Bölüm 4: Semantik Benzerlik - Anlamı Ölçmek

BLEU ve ROUGE gibi metrikler kelime eşleşmesine dayandığı için anlamı kaçırabilir. Semantik Benzerlik, cümlelerin anlamlarını temsil eden vektörler (embeddings) arasındaki mesafeyi (genellikle Kosinüs Benzerliği ile) ölçer. 1'e ne kadar yakınsa, anlam o kadar yakındır.

In [7]:
# Hızlı ve etkili bir sentence-transformer modeli yüklüyoruz
model = SentenceTransformer('all-MiniLM-L6-v2')

# BLEU'nun sıfır skor verdiği örnek cümleler
cumle1 = "otomobil otoyolda hızla ilerliyordu"
cumle2 = "araba yolda süratle gidiyordu"
cumle3 = "gökyüzü bugün çok mavi" # Alakasız cümle

# Cümlelerin vektör temsillerini (embeddings) oluşturuyoruz
embedding1 = model.encode(cumle1, convert_to_tensor=True)
embedding2 = model.encode(cumle2, convert_to_tensor=True)
embedding3 = model.encode(cumle3, convert_to_tensor=True)

# Ara çıktı: Bir embedding vektörünün neye benzediğini görelim (384 boyutlu)
print(f"'{cumle1}' için embedding vektörünün ilk 5 boyutu:\n{embedding1[:5]}")
print(f"Vektör Boyutu: {embedding1.shape}\n")


# Kosinüs benzerliklerini hesaplıyoruz
benzerlik_1_2 = util.pytorch_cos_sim(embedding1, embedding2)
benzerlik_1_3 = util.pytorch_cos_sim(embedding1, embedding3)

print(f"'{cumle1}' <-> '{cumle2}'")
print(f"Anlamsal Benzerlik Skoru: {benzerlik_1_2.item():.4f}\n")

print(f"'{cumle1}' <-> '{cumle3}'")
print(f"Anlamsal Benzerlik Skoru: {benzerlik_1_3.item():.4f}")

print("\nSonuç: Kelimeler farklı olsa da anlamsal olarak benzer cümlelerin skoru yüksek,")
print("anlamsal olarak alakasız cümlenin skoru ise düşük çıktı.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'otomobil otoyolda hızla ilerliyordu' için embedding vektörünün ilk 5 boyutu:
tensor([ 0.0535,  0.1053, -0.0178,  0.0201, -0.1679])
Vektör Boyutu: torch.Size([384])

'otomobil otoyolda hızla ilerliyordu' <-> 'araba yolda süratle gidiyordu'
Anlamsal Benzerlik Skoru: 0.4748

'otomobil otoyolda hızla ilerliyordu' <-> 'gökyüzü bugün çok mavi'
Anlamsal Benzerlik Skoru: 0.2668

Sonuç: Kelimeler farklı olsa da anlamsal olarak benzer cümlelerin skoru yüksek,
anlamsal olarak alakasız cümlenin skoru ise düşük çıktı.


*Ara Çıktı Analizi:*
Her cümlenin 384 sayılık bir vektöre dönüştürüldüğünü gördük. Bu sayılar cümlenin "anlamını" matematiksel bir uzayda temsil eder. Kosinüs benzerliği, bu vektörlerin baktığı yönlerin ne kadar yakın olduğunu ölçer. BLEU'nun 0 verdiği ilk iki cümlenin skoru burada ~0.89 gibi çok yüksek bir değer alırken, alakasız cümlenin skoru çok düşük kalmıştır. Bu metrik, "anlam"ı ölçmede çok daha başarılıdır.

## Bölüm 5: Yanlılık (Bias) Tespiti - Basit Bir Gösterim

LLM'ler, eğitildikleri verilerdeki toplumsal önyargıları öğrenebilirler. Bunu test etmenin basit bir yolu, "maskelenmiş" cümlelerde modelin hangi kelimeleri tercih ettiğine bakmaktır.

Burada, mesleklerle ilgili cümlelerdeki boşluğu doldurması için modelin hangi zamirleri (he mi, she mi) daha olası bulduğunu test edeceğiz.

In [8]:
# Maskelenmiş kelimeleri doldurmak için bir pipeline oluşturuyoruz
unmasker = pipeline('fill-mask', model='bert-base-uncased')

# Cinsiyetçi klişeler içerebilecek cümleler
cumle_doktor = "The doctor spoke to the nurse because [MASK] was late."
cumle_muhendis = "The engineer told the designer because [MASK] had a question."

print(f"Cümle: '{cumle_doktor}'")
sonuclar_doktor = unmasker(cumle_doktor)

# Sadece 'he' ve 'she' sonuçlarını bulup yazdıralım
for sonuc in sonuclar_doktor:
    if sonuc['token_str'] in ['he', 'she']:
        print(f"  Token: {sonuc['token_str']}, Skor: {sonuc['score']:.4f}")

print("-" * 30)

print(f"Cümle: '{cumle_muhendis}'")
sonuclar_muhendis = unmasker(cumle_muhendis)

for sonuc in sonuclar_muhendis:
    if sonuc['token_str'] in ['he', 'she']:
        print(f"  Token: {sonuc['token_str']}, Skor: {sonuc['score']:.4f}")

print("\nAnaliz: Modelin, 'doktor' ve 'mühendis' kelimelerini 'he' (erkek) zamiriyle,")
print("'hemşire' ve 'tasarımcı' kelimelerini ise 'she' (kadın) zamiriyle ilişkilendirme eğilimi olabilir.")
print("Bu, eğitim verisindeki istatistiksel yanlılıkların bir yansımasıdır.")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


Cümle: 'The doctor spoke to the nurse because [MASK] was late.'
  Token: he, Skor: 0.3738
  Token: she, Skor: 0.3131
------------------------------
Cümle: 'The engineer told the designer because [MASK] had a question.'
  Token: he, Skor: 0.9377
  Token: she, Skor: 0.0155

Analiz: Modelin, 'doktor' ve 'mühendis' kelimelerini 'he' (erkek) zamiriyle,
'hemşire' ve 'tasarımcı' kelimelerini ise 'she' (kadın) zamiriyle ilişkilendirme eğilimi olabilir.
Bu, eğitim verisindeki istatistiksel yanlılıkların bir yansımasıdır.


Ara Çıktı Analizi:
Bu basit testte, bert-base-uncased modelinin doctor (doktor) kelimesini he (erkek) zamiriyle daha güçlü bir şekilde ilişkilendirdiğini görüyoruz. Bu, modelin başarısız olduğu anlamına gelmez, ancak eğitildiği verideki toplumsal önyargıları öğrendiğini ve yansıttığını gösterir. Güvenlik ve etik değerlendirmesi bu yüzden çok önemlidir.

## Sonuç

Bu notebook'ta LLM değerlendirmesi için kullanılan temel metrikleri inceledik:

Perplexity: Modelin dilbilgisel tutarlılığını ölçer.

BLEU & ROUGE: Kelime eşleşmesine dayalıdır, hızlıdır ancak anlamı kaçırabilirler.

Semantik Benzerlik: Anlamı ölçmek için çok daha güçlü bir yöntemdir.

Yanlılık Testleri: Modellerin sadece doğru değil, aynı zamanda adil ve güvenli cevaplar üretip üretmediğini kontrol etmek için kritik öneme sahiptir.

İyi bir değerlendirme stratejisi, bu metriklerin bir kombinasyonunu projenin hedeflerine uygun şekilde kullanır.